In [19]:
# https://www.youtube.com/watch?v=u1vLJBwOFC8
"""
Artificial dataset created in Excel (anomaly.csv).
First column is Date
Second column is supposed to be Power reading: Good between 90 and 100
Third column: Detector reading: Good between 8-12
Last coumn is QUality which is either Good or Bad. 
"""


'\nArtificial dataset created in Excel (anomaly.csv).\nFirst column is Date\nSecond column is supposed to be Power reading: Good between 90 and 100\nThird column: Detector reading: Good between 8-12\nLast coumn is QUality which is either Good or Bad. \n'

In [20]:
import pandas as pd
df = pd.read_csv('anomaly.csv')

#print(df.columns) 
#print(df.head())
#df['Power'].plot(kind='hist', title='Power Reading', bins=30, figsize=(12,10)) 
#Most values between 90 and 100 with some outliers / anomalies
#df['Detector'].plot(kind='hist', title='Detector Reading', bins=30, figsize=(12,10)) 
#Most values between 8 and 12 with some outliers / anomalies

#To see how the data is spread betwen Good and Bad
print(df.groupby('Quality')['Quality'].count())

Quality
Bad      407
Good    1054
Name: Quality, dtype: int64


In [21]:
df.drop(['Date'], axis=1, inplace=True)
#print(df.head())

#If there are missing entries, drop them.
df.dropna(inplace=True,axis=1)


#COnvert non-numeric to numeric
df.Quality[df.Quality == 'Good'] = 1
df.Quality[df.Quality == 'Bad'] = 2


good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for good data points
#print(good_mask.head())

df.drop('Quality',axis=1,inplace=True)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [22]:
df_good = df[good_mask]/np.max(df[good_mask]) #veriyi normalize de ettik.Sonuçlar iyleşti. (orjinalde yoktu)
df_bad = df[bad_mask]/np.max(df[good_mask])

print(f"Good count: {len(df_good)}")
print(f"Bad count: {len(df_bad)}")

df_good[:5]
df_bad[:5]

Good count: 1054
Bad count: 407


,Power,Detector
0,0.96,0.666667
1,0.96,0.833333
2,0.91,0.666667
3,0.97,0.750000
4,0.91,0.916667


In [23]:
# This is the feature vector that goes to the neural net
x_good = df_good.values
x_bad = df_bad.values

from sklearn.model_selection import train_test_split

x_good_train, x_good_test = train_test_split(
        x_good, test_size=0.25, random_state=42)

print(f"Good train count: {len(x_good_train)}")
print(f"Good test count: {len(x_good_test)}")

Good train count: 790
Good test count: 264


In [24]:
######### 
#Define the autoencoder model
#Since we're dealing with numeric values we can use only Dense layers.
#burada veri numerik değerlerden oluştuğu için dense katmanları kullanıldı. 
#resim veri setlerinde ise dense yerine Conv2D kullanmak gerekir. 2D convolution layer açılımı olan bu
# katmanlar resmi uzamsal olarak evirir.Sonuçta tensörlerden (vektörler kümesi) çıktı üretir.

from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(x_good.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                30        
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
 dense_6 (Dense)             (None, 10)                40        
                                                                 
 dense_7 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(x_good_train,x_good_train,verbose=1,epochs=100)

Epoch 1/100
25/25 [==============================] - 1s 2ms/step - loss: 0.8217
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.7208
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6580
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5919
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.5204
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.4404
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.3500
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 0.2509
Epoch 9/100
25/25 [==============================] - 0s 3ms/step - loss: 0.1544
Epoch 10/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0837
Epoch 11/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0471
Epoch 12/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0312
Epoch 13/100
25/25 [=================

In [27]:
pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

print(f"Insample Good Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Good Score (RMSE): {score2}")
print(f"Bad sample Score (RMSE): {score3}")

Insample Good Score (RMSE): 0.024415441568331434
Out of Sample Good Score (RMSE): 0.024450867614233502
Bad sample Score (RMSE): 0.12227620123298866


Burada yapılan veride bulunan ve iyi kötü diye işaretlenmiş sayısal değerdeki verileri alıp iyi olanları kendisi ile eğittik. Daha sonra başka bir iyi olan veriyi girdi olarak verdiğimizde eğitim setine yakın bir başarı skoru elde ettik. Kötü olan veriyi verdiğimizde ise daha büyük başarısızlıkta bir skor elde ettik. Sonuç olarak bu şekilde bir verideki anomalileri tespit etmek mümkündür. Örneğin hastalıklı hücre fotoğrafları ile sağlıklı hücre fotoğrafları modelde eğitilerek sonuçta bir hücre resminden anomalileri tespit eden bir model geliştirilebilir.